# Catalogo datos.madrid con el pato

### Carga de datos en el pato
* dataset: **catalogo** 

In [ ]:
%%capture
import duckdb

#conexion a bbdd en memoria o en fichero
#conn = duckdb.connect() 
conn = duckdb.connect('/home/jcaubin/duck_catalogo.db') 

%load_ext sql
%sql conn --alias duckdb

#limite de filas a mostrar
%config SqlMagic.displaylimit = 100 

In [17]:
# imprime la hora de ejecución
from datetime import datetime
from IPython.display import display, Markdown

t = datetime.now()
display(Markdown(f"### hora de ejecución: {t}"))

### hora de ejecución: 2025-04-25 20:24:41.823466

In [12]:
import requests
import duckdb
import io

ds_url = 'https://datos.madrid.es/egob/catalogo.csv'

response = requests.get(ds_url)
if response.status_code==200:
    print(response.apparent_encoding)
    relation = duckdb.read_csv(io.StringIO(response.text), sep = ";", quotechar="(no escape)")

    # with open('../data/300392-11041819-meteorologia-tiempo-real.csv', 'wb') as tmp_file:
    #     tmp_file.write(response.content)
relation

ISO-8859-1


InvalidInputException: Invalid Input Error: The quote option cannot exceed a size of 1 byte.

In [19]:
%%sql 

create or replace table meteo_today as
select * from read_csv('../data/300392-11041819-meteorologia-tiempo-real.csv');

Running query in 'duckdb'

Count
94


In [20]:

%%sql

create or replace table estaciones as
select * from read_csv('https://datos.madrid.es/egob/catalogo/300360-1-meteorologicos-estaciones.csv')

Running query in 'duckdb'

Count
26


In [21]:

%%sql

create or replace table magnitudes as
select cast(CODIGO as int) CODIGO, PARAMETRO, UNIDAD, cast(COD_TECNICA as int) COD_TECNICA, TECNICA 
from read_csv('../maestras/parametros_meteo.csv')

Running query in 'duckdb'

Count
26


In [22]:
%%sql 

create or replace table td as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALOR	 from (
from meteo_today
unpivot (
    (VALOR)
for HORA in (
    H01, 
    H02, 
    H03, 
    H04, 
    H05, 
    H06, 
    H07, 
    H08, 
    H09, 
    H10, 
    H11, 
    H12, 
    H13, 
    H14, 
    H15, 
    H16, 
    H17, 
    H18, 
    H19, 
    H20, 
    H21, 
    H22, 
    H23, 
    H24 
    ) 
)
);

Running query in 'duckdb'

Count
2256


In [23]:
%%sql 

create or replace  table tv as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALIDEZ	 
from (
from meteo_today
unpivot (
    (VALIDEZ)
for HORA in (
    V01, 
    V02, 
    V03, 
    V04, 
    V05, 
    V06, 
    V07, 
    V08, 
    V09, 
    V10, 
    V11, 
    V12, 
    V13, 
    V14, 
    V15, 
    V16, 
    V17, 
    V18, 
    V19, 
    V20, 
    V21, 
    V22, 
    V23, 
    V24, 
    ) 
 ))
 ;


Running query in 'duckdb'

Count
2256


In [24]:
%%sql

select td.PROVINCIA, td.MUNICIPIO, td.ESTACION, td.MAGNITUD
    , concat(td.PROVINCIA, td.MUNICIPIO, td.ESTACION,  '_' ,td.MAGNITUD, '_98') PUNTO_MUESTREO, td.ANO, td.MES, td.DIA, td.H, td.VALOR 
    , tv.VALIDEZ, e.ESTACION ESTACION_DESC, e.ALTITUD, m.PARAMETRO
from td 
inner join tv on td.PROVINCIA=tv.PROVINCIA and 	td.MUNICIPIO=tv.MUNICIPIO and td.ESTACION=tv.ESTACION and 	td.MAGNITUD = tv.MAGNITUD
    and  td.ANO= tv.ANO and td.MES = tv.MES and TD.DIA = tv.DIA and td.H=tv.H
inner join estaciones e on td.estacion = e.CÓDIGO_CORTO
inner join magnitudes m on m.CODIGO = tv.MAGNITUD 

Running query in 'duckdb'

PROVINCIA,MUNICIPIO,ESTACION,MAGNITUD,PUNTO_MUESTREO,ANO,MES,DIA,H,VALOR,VALIDEZ,ESTACION_DESC,ALTITUD,PARAMETRO
28,079,102,81,28079102_81_98,2025,04,25,1,1.3,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,2,1.4,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,3,2.0,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,5,1.0,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,6,0.3,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,7,1.0,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,8,0.8,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,9,1.0,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,10,1.7,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO
28,079,102,81,28079102_81_98,2025,04,25,11,0.9,V,J.M.D. Moratalaz,686,VELOCIDAD VIENTO


In [25]:
%%sql 

create or replace table meteo_madrid24 as(
select td.PROVINCIA, td.MUNICIPIO, td.ESTACION, td.MAGNITUD
    , concat(td.PROVINCIA, td.MUNICIPIO, td.ESTACION,  '_' ,td.MAGNITUD, '_98') PUNTO_MUESTREO, td.ANO, td.MES, td.DIA, td.H, td.VALOR 
    , tv.VALIDEZ, e.ESTACION ESTACION_DESC, e.ALTITUD, m.PARAMETRO
from td 
inner join tv on td.PROVINCIA=tv.PROVINCIA and 	td.MUNICIPIO=tv.MUNICIPIO and td.ESTACION=tv.ESTACION and 	td.MAGNITUD = tv.MAGNITUD
    and  td.ANO= tv.ANO and td.MES = tv.MES and TD.DIA = tv.DIA and td.H=tv.H
inner join estaciones e on td.estacion = e.CÓDIGO_CORTO
inner join magnitudes m on m.CODIGO = tv.MAGNITUD 
)

Running query in 'duckdb'

Count
2256


In [26]:
%%sql
select punto_muestreo, count(1)
from calair
group by all
order by all

Running query in 'duckdb'

PUNTO_MUESTREO,count(1)
28079004_10_47,96
28079004_12_8,96
28079004_6_48,96
28079004_7_8,96
28079004_83_98,336
28079004_8_8,96
28079004_9_47,96
28079008_10_47,96
28079008_12_8,96
28079008_14_6,96


In [27]:
%%sql
/*MEGE de datos con la tabla principal*/
INSERT OR REPLACE INTO CALAIR BY NAME 
SELECT PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO, ANO, MES, DIA, H, VALOR, VALIDEZ, ESTACION_DESC, ALTITUD, PARAMETRO
FROM duck_test.main.meteo_madrid24;

Running query in 'duckdb'

Count
2256


In [28]:
%%sql

SELECT  ANO, MES, DIA, COUNT(1) n
FROM duck_test.main.calair
GROUP BY ALL
order by all;

Running query in 'duckdb'

ANO,MES,DIA,n
2025,01,13,2256
2025,01,14,2256
2025,01,15,2256
2025,01,16,2256
2025,01,17,2088
2025,01,19,2256
2025,01,20,2256
2025,01,21,2256
2025,01,22,2208
2025,01,23,2088


In [29]:
%sql --close duckdb